<a href="https://colab.research.google.com/github/mickymult/RAG-Mistral7b/blob/main/RAG_testing_mistral7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
# !pip install faiss-cpu
!pip install ctransformers
!nvidia-smi


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Sat Sep 13 19:43:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.88                 Driver Version: 576.88         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   39C    P8              1W /   80W |    2237MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [67]:
# from ctransformers import CTransformers

import os, glob, pathlib

p = r"C:\Models\llm\mistral-7b-instruct-v0.1.Q8_0.gguf"
print("Exists?", os.path.exists(p))
print("Is file?", os.path.isfile(p))
print("Dir listing:", glob.glob(r"C:\models\llm\*.gguf"))


Exists? True
Is file? True
Dir listing: ['C:\\models\\llm\\mistral-7b-instruct-v0.1.Q8_0.gguf']


In [68]:

def create_llms_model():
    # point directly to your local file path
    return CTransformers(
        model=p,
        model_type="mistral",               # important so tokenizer/arch is correct
        config={
            "max_new_tokens": 128,
            "temperature": 0.9,
            # "context_length": 1024,         # optional
            "gpu_layers": 8,                # CPU first; bump if you have GPU support
        },
    )
llm = create_llms_model()


In [69]:
# --- Simple test prompts ---
prompt1 = "Explain what a Large Language Model is in one sentence."
prompt2 = "What is 12 multiplied by 8?"
promet3 = "what is the token limmit for mistral 7b?"
promt4 = "how should i make a boy want to have sex with me?"


# print("Prompt 1:", prompt1)
# print("Response 1:", llm.invoke(prompt1))
# print("\nPrompt 2:", prompt2)
# print("Response 2:", llm.invoke(prompt2))

print("\nPrompt 3:", promet3)
print("Response 3:", llm.invoke(promet3))

print("\nPrompt 4:", promt4)
print("Response 4:", llm.invoke(promt4))
# Create memory


Prompt 3: what is the token limmit for mistral 7b?
Response 3: 

## Answer (1)

There are no hard limits on the number of tokens that can be used with the Mistral7B language model, it just depends on how much memory you have available to use as the tokenizer cache.

If you run out of memory while generating text, the model will stop producing output until more memory becomes available. You can control the amount of memory used by the model via environment variables like `TOKENIZER_MAX_CACHE_SIZE` and `TOKENIZER_CACHE_PATH`.

More generally you're probably better off using a larger language model with more tokens

Prompt 4: how should i make a boy want to have sex with me?
Response 4: 

i am 27 female i find all boys attractive. and there are times i will do whatever to get them to like me. but it is not working.

how can i make the male opposite attract him to me?

### Answer:

It's important to remember that sexual attraction is a complex thing and it cannot be forced or manipulated.

In [71]:
from langchain_community.llms import CTransformers
from pathlib import Path

# ==========================
# Model loader (offline)
# ==========================
p = r"C:\Models\llm\mistral-7b-instruct-v0.1.Q8_0.gguf"  # your GGUF path

def create_llms_model():
    if not Path(p).is_file():
        raise FileNotFoundError(f"Model file not found at {p}")
    return CTransformers(
        model=p,
        model_type="mistral",   # required so tokenizer/arch is correct
        config={
            "max_new_tokens": 128,
            "temperature": 0.9,  # make answers more creative
            "gpu_layers": 8,     # offload 8 layers to GPU (set 0 for CPU only)
        },
    )

llm = create_llms_model()



In [72]:
# ==========================
# Example 1: Getting Started
# ==========================
print("\n=== Example 1: Getting Started ===")
prompt1 = "Explain what a Large Language Model is in one sentence."
print("Prompt:", prompt1)
print("Response:", llm.invoke(prompt1))

# ==========================
# Example 2: Larger Prompt
# ==========================
print("\n=== Example 2: Larger Prompt ===")
long_text = (
    "Large Language Models are trained on vast amounts of data. "
    "They can answer questions, summarize text, and generate new content. "
    "This makes them useful for many applications in research, education, "
    "and software development. " * 5
)

prompt2 = f"Summarize this text in 3 bullet points:\n\n{long_text}"
print("Prompt:", prompt2[:120] + " ...")  # only print first 120 chars
print("Response:", llm.invoke(prompt2))

# ==========================
# Example 3: Several Stage Answer
# ==========================
print("\n=== Example 3: Several Stage Answer ===")
question = "Why might someone choose to run an LLM offline instead of online?"

plan = llm.invoke(f"Make a 3-step plan to answer this question:\n{question}")
print("\nPlan:", plan)

draft = llm.invoke(f"Write a short draft answer following this plan:\n{plan}")
print("\nDraft:", draft)

critique = llm.invoke(f"Give 2 improvements to make this draft better:\n{draft}")
print("\nCritique:", critique)

refined = llm.invoke(f"Rewrite the draft with these improvements:\nDraft:\n{draft}\n\nCritique:\n{critique}")
print("\nRefined:", refined)




=== Example 1: Getting Started ===
Prompt: Explain what a Large Language Model is in one sentence.
Response: 
A large language model is an artificial intelligence system designed to understand, generate, and respond to human language by processing vast amounts of data.

=== Example 2: Larger Prompt ===
Prompt: Summarize this text in 3 bullet points:

Large Language Models are trained on vast amounts of data. They can answer ques ...
Response: 

1. Large Language Models are trained on massive amounts of data
2. They can answer questions, summarize text, and generate new content
3. Useful for various applications in research, education, and software development

=== Example 3: Several Stage Answer ===

Plan: 

1. Consider the advantages and disadvantages of running LLMs offline and online. Advantages of running LLMs offline may include greater security, privacy, and independence from internet connectivity. Disadvantages could be a lack of scalability and real-time data availability. Con

In [75]:
!pip install ollama


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [76]:
# ==========================
# Example 4: Online Integration (Ollama)
# ==========================
print("\n=== Example 4: Online Integration (Ollama) ===")

try:
    import ollama  # pip install ollama (but you need the Ollama app running locally)

    # Pick one of the free models that Ollama can pull automatically:
    # "llama3", "mistral", "gemma:2b", "phi3", "qwen2:1.5b"
    online_model = "llama3"
    online_prompt = "In one line, compare online vs offline use of LLMs."

    response = ollama.chat(
        model=online_model,
        messages=[{"role": "user", "content": online_prompt}],
    )

    print(f"Ollama ({online_model}) Response:", response["message"]["content"].strip())

except ImportError:
    print("Ollama package not installed → run `pip install ollama`.")
except Exception as e:
    print("Ollama call failed:", e)



=== Example 4: Online Integration (Ollama) ===
Ollama call failed: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download
